<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analysis and Prediction of HDB Resale Flat Price

## Contents:
- [Problem Statement](#Problem-Statement)
- [Data Import and Cleaning](#Data-Import-and-Cleaning)
- [Feature Engineering](#Feature-Engineering)

## Problem Statement

With rising resale flat prices, affordable public housing has become a concern to new homeowners. According to an annual wellness survey conducted by OCBC bank, 56% of Singaporeans who are in their 20s are worried that they cannot afford a home. 

As many aspiring homeowners will be financing their homes using their CPF savings, knowing the price of the HDB resale flat that they wish to purchase can help them better plan their finances and CPF contributions. 

So, how can the CPF Board assist these potential homeowners?

This project explores the key features that affect HDB resale prices in Singapore using data science methodologies. The objective is to build and deploy a reliable HDB resale price calculator that is intuitive and easy for potential homeowners to use, to help them understand what could be the resale price of their desired flat based on the key features. 

## Data Import and Cleaning

In [1]:
# Import the required packages for cleaning
import pandas as pd
import numpy as np
import math

In [2]:
# Set the display option to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Read in the train dataset
df = pd.read_csv("../datasets/train.csv")

/var/folders/6v/8flxs1q922d7hx42gvpwp7dm0000gn/T/ipykernel_24526/3936474927.py:2: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../datasets/train.csv")


In [4]:
# Show the information of the trained dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 78 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         150634 non-null  int64  
 1   Tranc_YearMonth            150634 non-null  object 
 2   town                       150634 non-null  object 
 3   flat_type                  150634 non-null  object 
 4   block                      150634 non-null  object 
 5   street_name                150634 non-null  object 
 6   storey_range               150634 non-null  object 
 7   floor_area_sqm             150634 non-null  float64
 8   flat_model                 150634 non-null  object 
 9   lease_commence_date        150634 non-null  int64  
 10  resale_price               150634 non-null  float64
 11  Tranc_Year                 150634 non-null  int64  
 12  Tranc_Month                150634 non-null  int64  
 13  mid_storey                 15

In [5]:
# Find the columns with null values
df.isnull().sum().sort_values(ascending=False)

Hawker_Within_500m           97390
Mall_Within_500m             92789
Hawker_Within_1km            60868
Hawker_Within_2km            29202
Mall_Within_1km              25426
Mall_Within_2km               1940
Mall_Nearest_Distance          829
Hawker_Nearest_Distance          0
mrt_nearest_distance             0
hawker_market_stalls             0
hawker_food_stalls               0
id                               0
bus_interchange                  0
planning_area                    0
Longitude                        0
Latitude                         0
postal                           0
other_room_rental                0
mrt_name                         0
mrt_interchange                  0
Tranc_YearMonth                  0
pri_sch_affiliation              0
sec_sch_latitude                 0
affiliation                      0
cutoff_point                     0
sec_sch_name                     0
sec_sch_nearest_dist             0
pri_sch_longitude                0
pri_sch_latitude    

In [6]:
# Check data for Mall_Nearest_Distance is null
df[pd.isnull(df["Mall_Nearest_Distance"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
75,163384,2021-04,SENGKANG,4 ROOM,468A,FERNVALE LINK,19 TO 21,93.0,Model A,2016,455000.0,2021,4,20,19,21,20,4 ROOM Model A,"468A, FERNVALE LINK",1001.052,454.521843,5,26,2015,Y,N,N,N,N,100,0,0,0,50,50,0,0,0,0,0,0,0,791468,1.397479,103.878500,Sengkang,NaN,NaN,NaN,NaN,2541.170822,NaN,NaN,NaN,40,0,1996.136647,Sengkang,1,1,1.392440,103.895735,162.723867,AFT BLK 467B,1.398494,103.877445,190.740687,Fern Green Primary School,83,0,1.397621,103.880210,789.399776,Pei Hwa Secondary School,219,0,1.390380,103.878593
321,163733,2021-03,CHOA CHU KANG,4 ROOM,815C,CHOA CHU KANG AVE 7,19 TO 21,93.0,Model A,2017,495000.0,2021,3,20,19,21,20,4 ROOM Model A,"815C, CHOA CHU KANG AVE 7",1001.052,494.479807,4,20,2016,Y,N,N,N,N,190,0,0,0,95,95,0,0,0,0,0,0,0,683815,1.376304,103.747458,Choa Chu Kang,NaN,NaN,NaN,NaN,2787.822081,NaN,NaN,NaN,28,45,1106.769957,Choa Chu Kang,1,1,1.385711,103.744203,98.827711,Opp Hong San Terr,1.377178,103.747622,607.693266,South View Primary School,38,0,1.381767,103.747290,926.560287,Chua Chu Kang Secondary School,211,0,1.383275,103.752025
478,68334,2021-02,GEYLANG,3 ROOM,18B,CIRCUIT RD,01 TO 03,67.0,Model A,2016,425000.0,2021,2,2,1,3,2,3 ROOM Model A,"18B, CIRCUIT RD",721.188,589.305424,5,19,2015,Y,Y,N,N,N,120,0,0,52,68,0,0,0,0,0,0,0,0,372018,1.329181,103.888248,Geylang,NaN,NaN,NaN,NaN,186.841799,2.0,4.0,7.0,16,43,456.647572,MacPherson,0,1,1.325859,103.890663,152.182792,Blk 36,1.328862,103.886917,780.318521,Canossa Catholic Primary School,47,1,1.326511,103.881757,1366.262344,Bartley Secondary School,188,0,1.340017,103.882455
643,68382,2021-02,PUNGGOL,4 ROOM,122A,EDGEDALE PLAINS,10 TO 12,93.0,Model A,2017,455000.0,2021,2,11,10,12,11,4 ROOM Model A,"122A, EDGEDALE PLAINS",1001.052,454.521843,4,17,2016,Y,Y,N,N,N,144,0,0,32,80,32,0,0,0,0,0,0,0,821122,1.393510,103.909107,Punggol,NaN,NaN,NaN,NaN,3562.641627,NaN,NaN,NaN,40,0,1481.381271,Punggol,1,1,1.405157,103.902636,276.390975,Blk 156A CP,1.392005,103.907128,160.776237,Mee Toh School,93,1,1.394854,103.908573,501.165620,Greendale Secondary School,203,0,1.397281,103.911576
691,68193,2020-12,CLEMENTI,3 ROOM,440A,CLEMENTI AVE 3,10 TO 12,69.0,Model A,2018,598000.0,2020,12,11,10,12,11,3 ROOM Model A,"440A, CLEMENTI AVE 3",742.716,805.152979,3,30,2017,Y,N,N,N,N,208,0,54,102,52,0,0,0,0,0,0,0,0,121440,1.315999,103.764183,Clementi,NaN,NaN,NaN,NaN,296.393771,1.0,4.0,5.0,51,105,110.765788,Clementi,1,0,1.315406,103.764983,120.673101,Clementi Int,1.314916,103.764122,127.150007,Clementi Primary School,75,0,1.315809,103.763055,241.034822,Clementi Town Secondary School,231,0,1.315475,103.762079


Mall_Nearest_Distance with null values also have null values for Mall_Within_500m, Mall_Within_1km and Mall_Within_2km. This could indicate data entry error or that there are no malls within 2km of the flat. These account for 0.5% of the dataframe, hence decision is to drop these rows. 

In [7]:
# Drop rows where Mall_Nearest_Distance is null
df.dropna(subset = "Mall_Nearest_Distance", inplace = True)

In [8]:
# Check data for Mall_Nearest_distance equal 0
df[df["Mall_Nearest_Distance"]==0].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
443,114880,2019-05,WOODLANDS,5 ROOM,680,WOODLANDS AVE 6,10 TO 12,123.0,Improved,1998,479888.0,2019,5,11,10,12,11,5 ROOM Improved,"680, WOODLANDS AVE 6",1323.972,362.460838,23,13,1997,Y,N,N,N,N,96,0,0,0,48,48,0,0,0,0,0,0,0,730680,1.439633,103.802122,Woodlands,0.0,1.0,3.0,5.0,163.411073,1.0,1.0,1.0,43,0,149.102116,Admiralty,0,0,1.440343,103.800984,44.555637,Bef Admiralty Stn/Blk 680,1.439739,103.802508,283.774985,Greenwood Primary School,61,0,1.440110,103.804629,429.863620,Woodlands Ring Secondary School,211,0,1.436114,103.800521
3533,106391,2012-03,CHOA CHU KANG,5 ROOM,137,TECK WHYE LANE,01 TO 05,120.0,Improved,1989,430000.0,2012,3,3,1,5,3,5 ROOM Improved,"137, TECK WHYE LANE",1291.680,332.899789,32,6,1989,Y,Y,N,N,N,39,0,0,0,18,21,0,0,0,0,0,0,0,680137,1.380048,103.752465,Choa Chu Kang,0.0,1.0,4.0,10.0,2230.808514,NaN,NaN,NaN,28,45,1057.441996,Bukit Panjang,1,1,1.379016,103.761921,189.439711,Blk 108,1.378526,103.753230,413.180046,Teck Whye Primary School,73,0,1.383558,103.753683,362.158091,Chua Chu Kang Secondary School,211,0,1.383275,103.752025
10071,164453,2015-01,JURONG WEST,5 ROOM,221,BOON LAY PL,19 TO 21,109.0,Standard,1979,418000.0,2015,1,20,19,21,20,5 ROOM Standard,"221, BOON LAY PL",1173.276,356.267409,42,25,1978,Y,Y,N,N,N,84,0,0,0,0,84,0,0,0,0,0,0,0,640221,1.346374,103.712473,Jurong West,0.0,1.0,1.0,5.0,128.107132,1.0,2.0,3.0,72,94,988.563551,Lakeside,0,0,1.344075,103.721063,65.485775,Boon Lay Shop Ctr,1.346398,103.713061,396.050713,Boon Lay Garden Primary School,90,0,1.342847,103.712969,445.049959,River Valley High School,250,1,1.344093,103.709183
11695,106388,2012-08,CHOA CHU KANG,4 ROOM,137,TECK WHYE LANE,04 TO 06,103.0,Model A,1989,388000.0,2012,8,5,4,6,5,4 ROOM Model A,"137, TECK WHYE LANE",1108.692,349.961937,32,6,1989,Y,Y,N,N,N,39,0,0,0,18,21,0,0,0,0,0,0,0,680137,1.380048,103.752465,Choa Chu Kang,0.0,1.0,4.0,10.0,2230.808514,NaN,NaN,NaN,28,45,1057.441996,Bukit Panjang,1,1,1.379016,103.761921,189.439711,Blk 108,1.378526,103.753230,413.180046,Teck Whye Primary School,73,0,1.383558,103.753683,362.158091,Chua Chu Kang Secondary School,211,0,1.383275,103.752025
32075,106387,2018-11,CHOA CHU KANG,4 ROOM,137,TECK WHYE LANE,04 TO 06,106.0,Model A,1989,315000.0,2018,11,5,4,6,5,4 ROOM Model A,"137, TECK WHYE LANE",1140.984,276.077491,32,6,1989,Y,Y,N,N,N,39,0,0,0,18,21,0,0,0,0,0,0,0,680137,1.380048,103.752465,Choa Chu Kang,0.0,1.0,4.0,10.0,2230.808514,NaN,NaN,NaN,28,45,1057.441996,Bukit Panjang,1,1,1.379016,103.761921,189.439711,Blk 108,1.378526,103.753230,413.180046,Teck Whye Primary School,73,0,1.383558,103.753683,362.158091,Chua Chu Kang Secondary School,211,0,1.383275,103.752025


Mall_Nearest_Distance = 0 actually has malls within 2km of the flat, hence it is unlikely to be distance of 0. These account for 0.02% of the dataframe, hence decision is to drop these rows.

In [9]:
# Drop the rows where Mall_Nearest_Distance' is 0
df = df[df["Mall_Nearest_Distance"] != 0]

Check the rest of the columns with null values

In [10]:
# Check data for Mall_Within_500m is null
df[pd.isnull(df["Mall_Within_500m"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,680000.0,2016,5,11,10,12,11,4 ROOM Model A,"3B, UPP BOON KENG RD",968.760,701.928238,15,25,2005,Y,N,N,N,N,142,0,0,0,96,46,0,0,0,0,0,0,0,382003,1.314299,103.872828,Kallang,1094.090418,NaN,NaN,7.0,154.753357,1.0,3.0,13.0,84,60,330.083069,Kallang,0,0,1.311540,103.871731,29.427395,Blk 3B,1.314433,103.872600,1138.633422,Geylang Methodist School,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,665000.0,2012,7,8,7,9,8,5 ROOM Improved,"153, BISHAN ST 13",1399.320,475.230826,34,9,1987,Y,N,N,N,N,112,0,0,0,56,56,0,0,0,0,0,0,0,570153,1.346086,103.855078,Bishan,866.941448,NaN,1.0,3.0,640.151925,NaN,1.0,7.0,80,77,903.659703,Bishan,1,1,1.350580,103.848305,58.207761,BLK 151A MKT,1.345659,103.855381,415.607357,Kuo Chuan Presbyterian Primary School,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,838000.0,2013,7,14,13,15,14,EXECUTIVE Apartment,"289B, BT BATOK ST 25",1550.016,540.639580,24,16,1996,Y,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,651289,1.343867,103.760535,Bukit Batok,1459.579948,NaN,NaN,4.0,1762.082341,NaN,NaN,1.0,84,95,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,Blk 289E,1.344064,103.758613,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,550000.0,2012,4,3,1,5,3,4 ROOM Model A,"232, BISHAN ST 22",1108.692,496.080066,29,11,1990,Y,Y,N,N,N,75,0,0,0,68,7,0,0,0,0,0,0,0,570232,1.358245,103.845504,Bishan,950.175199,NaN,1.0,4.0,726.215262,NaN,1.0,9.0,32,86,907.453484,Bishan,1,1,1.350580,103.848305,43.396521,Opp Bishan Nth Shop Mall,1.358045,103.845169,389.515528,Catholic High School,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,298000.0,2017,12,2,1,3,2,4 ROOM Simplified,"876, YISHUN ST 81",893.412,333.552717,34,4,1987,Y,N,N,N,N,48,0,0,0,30,18,0,0,0,0,0,0,0,760876,1.414745,103.835532,Yishun,729.771895,NaN,1.0,2.0,1540.151439,NaN,NaN,1.0,45,0,412.343032,Khatib,0,0,1.417131,103.832692,129.422752,Blk 873,1.415424,103.836477,401.200584,Naval Base Primary School,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


In [11]:
# Check data for Mall_Within_1km is null
df[pd.isnull(df["Mall_Within_1km"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,680000.0,2016,5,11,10,12,11,4 ROOM Model A,"3B, UPP BOON KENG RD",968.760,701.928238,15,25,2005,Y,N,N,N,N,142,0,0,0,96,46,0,0,0,0,0,0,0,382003,1.314299,103.872828,Kallang,1094.090418,NaN,NaN,7.0,154.753357,1.0,3.0,13.0,84,60,330.083069,Kallang,0,0,1.311540,103.871731,29.427395,Blk 3B,1.314433,103.872600,1138.633422,Geylang Methodist School,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,838000.0,2013,7,14,13,15,14,EXECUTIVE Apartment,"289B, BT BATOK ST 25",1550.016,540.639580,24,16,1996,Y,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,651289,1.343867,103.760535,Bukit Batok,1459.579948,NaN,NaN,4.0,1762.082341,NaN,NaN,1.0,84,95,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,Blk 289E,1.344064,103.758613,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
7,55093,2012-03,BEDOK,3 ROOM,701,BEDOK RESERVOIR RD,06 TO 10,68.0,New Generation,1980,340000.0,2012,3,8,6,10,8,3 ROOM New Generation,"701, BEDOK RESERVOIR RD",731.952,464.511334,41,12,1980,Y,Y,N,N,N,132,0,0,110,22,0,0,0,0,0,0,0,0,470701,1.337642,103.918906,Bedok,1749.147519,NaN,NaN,3.0,734.252454,NaN,2.0,5.0,36,75,291.626388,Bedok North,0,0,1.335166,103.918039,71.972918,Opp Blk 701,1.338071,103.919390,365.790881,Damai Primary School,56,0,1.335351,103.921268,835.014747,Damai Secondary School,188,0,1.337545,103.926417
27,69679,2020-06,BUKIT BATOK,4 ROOM,291A,BT BATOK ST 24,01 TO 03,104.0,Model A,1997,360000.0,2020,6,2,1,3,2,4 ROOM Model A,"291A, BT BATOK ST 24",1119.456,321.584770,24,19,1996,Y,N,N,N,N,126,0,0,0,90,36,0,0,0,0,0,0,0,650291,1.343668,103.756052,Bukit Batok,1063.974768,NaN,NaN,4.0,2044.044484,NaN,NaN,NaN,60,87,941.548574,Bukit Batok,1,0,1.349561,103.749970,36.778335,Blk 291A,1.343995,103.756100,176.985353,Keming Primary School,39,0,1.345245,103.756265,464.674132,Yusof Ishak Secondary School,188,0,1.342334,103.760013
35,56979,2018-11,KALLANG/WHAMPOA,3 ROOM,64,KALLANG BAHRU,01 TO 03,65.0,Improved,1974,260000.0,2018,11,2,1,3,2,3 ROOM Improved,"64, KALLANG BAHRU",699.660,371.609067,47,13,1973,Y,N,N,N,N,284,0,0,280,0,4,0,0,0,0,0,0,0,330064,1.320347,103.868347,Kallang,1211.930580,NaN,NaN,6.0,232.225701,1.0,3.0,11.0,84,127,379.231503,Geylang Bahru,0,0,1.321301,103.871623,170.231691,Opp Blk 66,1.319965,103.869830,392.171154,Bendemeer Primary School,56,0,1.322208,103.865351,947.311902,Bendemeer Secondary School,188,0,1.324611,103.860970


In [12]:
# Check data for Mall_Within_2km is null
df[pd.isnull(df["Mall_Within_2km"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
109,156373,2013-01,BEDOK,5 ROOM,601,BEDOK RESERVOIR RD,07 TO 09,121.0,Improved,1982,510000.0,2013,1,8,7,9,8,5 ROOM Improved,"601, BEDOK RESERVOIR RD",1302.444,391.571538,39,12,1982,Y,N,N,N,N,132,0,0,0,20,112,0,0,0,0,0,0,0,470601,1.330067,103.911619,Bedok,2050.201309,NaN,NaN,NaN,432.094898,1.0,1.0,3.0,36,75,634.008786,Kaki Bukit,0,0,1.335322,103.909406,220.932249,Blk 609,1.332015,103.912011,98.777338,Telok Kurau Primary School,66,0,1.330913,103.911349,1075.003159,Ping Yi Secondary School,189,0,1.32714,103.920836
220,85651,2014-08,BEDOK,4 ROOM,123,BEDOK RESERVOIR RD,10 TO 12,84.0,Simplified,1987,370000.0,2014,8,11,10,12,11,4 ROOM Simplified,"123, BEDOK RESERVOIR RD",904.176,409.212366,34,12,1986,Y,Y,N,N,N,88,0,0,0,88,0,0,0,0,0,0,0,0,470123,1.332320,103.910395,Bedok,2247.327448,NaN,NaN,NaN,430.542469,1.0,1.0,3.0,36,75,351.506785,Kaki Bukit,0,0,1.335322,103.909406,86.157072,Blk 122,1.331779,103.909839,188.934321,Telok Kurau Primary School,66,0,1.330913,103.911349,1093.318715,Manjusri Secondary School,188,0,1.32752,103.901811
662,122577,2012-03,BEDOK,5 ROOM,671,JLN DAMAI,01 TO 05,127.0,Improved,1996,523000.0,2012,3,3,1,5,3,5 ROOM Improved,"671, JLN DAMAI",1367.028,382.581776,25,9,1995,Y,N,N,N,N,64,0,0,0,48,16,0,0,0,0,0,0,0,411671,1.332874,103.909045,Bedok,2218.607002,NaN,NaN,NaN,574.651057,NaN,1.0,3.0,36,75,275.120685,Kaki Bukit,0,0,1.335322,103.909406,150.429958,Blk 122,1.331779,103.909839,336.378755,Telok Kurau Primary School,66,0,1.330913,103.911349,1000.570068,Manjusri Secondary School,188,0,1.32752,103.901811
672,146870,2014-12,BEDOK,4 ROOM,602,BEDOK RESERVOIR RD,10 TO 12,98.0,New Generation,1982,398000.0,2014,12,11,10,12,11,4 ROOM New Generation,"602, BEDOK RESERVOIR RD",1054.872,377.296961,39,13,1981,Y,N,N,N,N,120,0,0,0,24,96,0,0,0,0,0,0,0,470602,1.329349,103.911583,Bedok,2038.439070,NaN,NaN,NaN,496.490066,1.0,1.0,3.0,36,75,706.907914,Kaki Bukit,0,0,1.335322,103.909406,197.877475,Grosvenor View,1.327656,103.912131,175.898516,Telok Kurau Primary School,66,0,1.330913,103.911349,1057.480537,Ping Yi Secondary School,189,0,1.32714,103.920836
798,85682,2019-08,BEDOK,4 ROOM,132,BEDOK RESERVOIR RD,04 TO 06,103.0,Model A,1987,410000.0,2019,8,5,4,6,5,4 ROOM Model A,"132, BEDOK RESERVOIR RD",1108.692,369.805140,34,12,1986,Y,N,N,N,N,88,0,0,0,88,0,0,0,0,0,0,0,0,470132,1.332795,103.911813,Bedok,2115.704584,NaN,NaN,NaN,267.241208,1.0,1.0,3.0,36,75,387.994443,Kaki Bukit,0,0,1.335322,103.909406,67.211243,Blk 133,1.332192,103.911841,215.532404,Telok Kurau Primary School,66,0,1.330913,103.911349,1183.818023,Ping Yi Secondary School,189,0,1.32714,103.920836


In [13]:
# Check data for Hawker_Within_500m is null
df[pd.isnull(df["Hawker_Within_500m"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,665000.0,2012,7,8,7,9,8,5 ROOM Improved,"153, BISHAN ST 13",1399.320,475.230826,34,9,1987,Y,N,N,N,N,112,0,0,0,56,56,0,0,0,0,0,0,0,570153,1.346086,103.855078,Bishan,866.941448,NaN,1.0,3.0,640.151925,NaN,1.0,7.0,80,77,903.659703,Bishan,1,1,1.350580,103.848305,58.207761,BLK 151A MKT,1.345659,103.855381,415.607357,Kuo Chuan Presbyterian Primary School,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,838000.0,2013,7,14,13,15,14,EXECUTIVE Apartment,"289B, BT BATOK ST 25",1550.016,540.639580,24,16,1996,Y,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,651289,1.343867,103.760535,Bukit Batok,1459.579948,NaN,NaN,4.0,1762.082341,NaN,NaN,1.0,84,95,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,Blk 289E,1.344064,103.758613,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,550000.0,2012,4,3,1,5,3,4 ROOM Model A,"232, BISHAN ST 22",1108.692,496.080066,29,11,1990,Y,Y,N,N,N,75,0,0,0,68,7,0,0,0,0,0,0,0,570232,1.358245,103.845504,Bishan,950.175199,NaN,1.0,4.0,726.215262,NaN,1.0,9.0,32,86,907.453484,Bishan,1,1,1.350580,103.848305,43.396521,Opp Bishan Nth Shop Mall,1.358045,103.845169,389.515528,Catholic High School,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,298000.0,2017,12,2,1,3,2,4 ROOM Simplified,"876, YISHUN ST 81",893.412,333.552717,34,4,1987,Y,N,N,N,N,48,0,0,0,30,18,0,0,0,0,0,0,0,760876,1.414745,103.835532,Yishun,729.771895,NaN,1.0,2.0,1540.151439,NaN,NaN,1.0,45,0,412.343032,Khatib,0,0,1.417131,103.832692,129.422752,Blk 873,1.415424,103.836477,401.200584,Naval Base Primary School,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335
6,173300,2018-05,HOUGANG,5 ROOM,641,HOUGANG AVE 8,07 TO 09,120.0,Improved,1987,433000.0,2018,5,8,7,9,8,5 ROOM Improved,"641, HOUGANG AVE 8",1291.680,335.222346,34,17,1986,Y,N,N,N,N,64,0,0,0,0,64,0,0,0,0,0,0,0,530641,1.371276,103.880059,Hougang,490.262931,1.0,1.0,8.0,516.981824,NaN,1.0,4.0,40,0,1370.648274,Kovan,0,0,1.359703,103.884303,46.567429,Blk 639,1.370877,103.880187,331.669318,Xinmin Primary School,48,0,1.371415,103.883039,294.826501,Bowen Secondary School,224,0,1.370751,103.877459


In [14]:
# Check data for Hawker_Within_1km is null
df[pd.isnull(df["Hawker_Within_1km"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,838000.0,2013,7,14,13,15,14,EXECUTIVE Apartment,"289B, BT BATOK ST 25",1550.016,540.639580,24,16,1996,Y,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,651289,1.343867,103.760535,Bukit Batok,1459.579948,NaN,NaN,4.0,1762.082341,NaN,NaN,1.0,84,95,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,Blk 289E,1.344064,103.758613,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,298000.0,2017,12,2,1,3,2,4 ROOM Simplified,"876, YISHUN ST 81",893.412,333.552717,34,4,1987,Y,N,N,N,N,48,0,0,0,30,18,0,0,0,0,0,0,0,760876,1.414745,103.835532,Yishun,729.771895,NaN,1.0,2.0,1540.151439,NaN,NaN,1.0,45,0,412.343032,Khatib,0,0,1.417131,103.832692,129.422752,Blk 873,1.415424,103.836477,401.200584,Naval Base Primary School,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335
8,166738,2020-01,BISHAN,5 ROOM,450,SIN MING AVE,10 TO 12,125.0,Improved,1989,700000.0,2020,1,11,10,12,11,5 ROOM Improved,"450, SIN MING AVE",1345.500,520.252694,32,25,1988,Y,N,N,N,N,92,0,0,0,0,84,8,0,0,0,0,0,0,570450,1.358356,103.830398,Bishan,445.717070,1.0,1.0,1.0,1032.848455,NaN,NaN,4.0,31,51,1584.098301,Marymount,0,0,1.347873,103.840048,105.182580,Blk 454,1.359298,103.830321,398.373554,Ai Tong School,30,0,1.360736,103.833076,866.362958,Peirce Secondary School,208,0,1.366132,103.829910
9,103385,2014-06,SENGKANG,5 ROOM,307D,ANCHORVALE RD,10 TO 12,110.0,Improved,2001,490000.0,2014,6,11,10,12,11,5 ROOM Improved,"307D, ANCHORVALE RD",1184.040,413.837370,20,17,2000,Y,N,N,N,N,96,0,0,0,32,64,0,0,0,0,0,0,0,544307,1.390299,103.886489,Sengkang,853.383254,NaN,2.0,7.0,1743.704293,NaN,NaN,1.0,40,0,1054.995148,Sengkang,1,1,1.392440,103.895735,89.350913,Blk 309D,1.390440,103.885698,94.340507,Anchor Green Primary School,75,0,1.390662,103.887256,458.532781,Nan Chiau High School,243,0,1.388687,103.890286
10,90310,2013-06,SENGKANG,4 ROOM,273D,COMPASSVALE LINK,13 TO 15,90.0,Premium Apartment,2009,625000.0,2013,6,14,13,15,14,4 ROOM Premium Apartment,"273D, COMPASSVALE LINK",968.760,645.154631,12,16,2008,Y,N,N,N,N,112,0,0,0,112,0,0,0,0,0,0,0,0,544273,1.384141,103.894443,Sengkang,835.381881,NaN,3.0,8.0,1634.750527,NaN,NaN,1.0,40,0,198.410931,Buangkok,1,0,1.382650,103.893463,157.722157,Buangkok Stn Exit A,1.383933,103.893040,209.388465,North Vista Primary School,79,0,1.382893,103.895854,490.102280,Seng Kang Secondary School,199,0,1.386099,103.898393


In [15]:
# Check data for Hawker_Within_2km is null
df[pd.isnull(df["Hawker_Within_2km"])].head()

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
26,166009,2019-08,JURONG WEST,5 ROOM,653A,JURONG WEST ST 61,04 TO 06,111.0,Improved,2002,453000.0,2019,8,5,4,6,5,5 ROOM Improved,"653A, JURONG WEST ST 61",1194.804,379.141683,19,16,2000,Y,N,N,N,N,152,0,0,0,0,152,0,0,0,0,0,0,0,641653,1.336891,103.696639,Jurong West,521.399540,NaN,1.0,3.0,2028.864896,NaN,NaN,NaN,72,94,75.259191,Pioneer,0,0,1.337343,103.697143,57.023229,Blk 653B,1.336491,103.696319,339.509335,Frontier Primary School,34,0,1.336643,103.699683,633.528960,Jurong West Secondary School,199,0,1.335256,103.702098
27,69679,2020-06,BUKIT BATOK,4 ROOM,291A,BT BATOK ST 24,01 TO 03,104.0,Model A,1997,360000.0,2020,6,2,1,3,2,4 ROOM Model A,"291A, BT BATOK ST 24",1119.456,321.584770,24,19,1996,Y,N,N,N,N,126,0,0,0,90,36,0,0,0,0,0,0,0,650291,1.343668,103.756052,Bukit Batok,1063.974768,NaN,NaN,4.0,2044.044484,NaN,NaN,NaN,60,87,941.548574,Bukit Batok,1,0,1.349561,103.749970,36.778335,Blk 291A,1.343995,103.756100,176.985353,Keming Primary School,39,0,1.345245,103.756265,464.674132,Yusof Ishak Secondary School,188,0,1.342334,103.760013
30,116245,2016-03,CHOA CHU KANG,4 ROOM,417,CHOA CHU KANG AVE 4,07 TO 09,105.0,Model A,1993,383000.0,2016,3,8,7,9,8,4 ROOM Model A,"417, CHOA CHU KANG AVE 4",1130.220,338.872078,28,15,1991,Y,N,N,N,N,90,0,0,0,64,26,0,0,0,0,0,0,0,680417,1.382965,103.741578,Choa Chu Kang,469.200578,1.0,3.0,7.0,3472.690804,NaN,NaN,NaN,28,45,422.330153,Choa Chu Kang,1,1,1.385711,103.744203,69.897198,Bet Blks 416/417,1.383534,103.741846,581.474674,Chua Chu Kang Primary School,68,0,1.377743,103.741861,393.733430,Bukit Panjang Government High School,244,0,1.382137,103.738134
31,108393,2012-11,SEMBAWANG,5 ROOM,507C,WELLINGTON CIRCLE,10 TO 12,110.0,Improved,2001,503000.0,2012,11,11,10,12,11,5 ROOM Improved,"507C, WELLINGTON CIRCLE",1184.040,424.816729,20,12,2000,Y,N,N,N,N,113,0,0,0,47,66,0,0,0,0,0,0,0,753507,1.451731,103.823452,Sembawang,589.343255,NaN,1.0,2.0,2278.363660,NaN,NaN,NaN,56,123,460.630740,Sembawang,1,0,1.448927,103.820402,99.783404,Blk 506A,1.452059,103.824288,127.755384,Wellington Primary School,85,0,1.451934,103.822321,1013.080578,Canberra Secondary School,188,0,1.453280,103.814471
33,113435,2012-07,SENGKANG,5 ROOM,299B,COMPASSVALE ST,13 TO 15,115.0,Premium Apartment,2001,496000.0,2012,7,14,13,15,14,5 ROOM Premium Apartment,"299B, COMPASSVALE ST",1237.860,400.691516,20,17,1999,Y,N,N,N,N,177,0,0,0,51,94,32,0,0,0,0,0,0,542299,1.396829,103.901187,Sengkang,627.487761,NaN,2.0,6.0,3159.026726,NaN,NaN,NaN,40,0,778.078440,Sengkang,1,1,1.392440,103.895735,226.977472,Blk 298A,1.394901,103.901859,449.115436,Compassvale Primary School,72,0,1.394530,103.897865,161.155684,Compassvale Secondary School,210,0,1.396218,103.899872


Comparing the above null columns with the mall/hawker nearest distance, the null values indicated there are no malls within the respective distance, hence we will inpute these null values as 0. 

In [16]:
# Impute null values in specific columns with 0

# List of columns to impute
columns_to_impute = ["Mall_Within_500m", "Mall_Within_1km","Mall_Within_2km", "Hawker_Within_500m", "Hawker_Within_1km", "Hawker_Within_2km"]

# Impute null values with 0 in the specified columns
df[columns_to_impute] = df[columns_to_impute].fillna(value = 0, axis = 1)

In [17]:
# Recheck that there are no more null values after cleaning
df.isnull().sum()

id                           0
Tranc_YearMonth              0
town                         0
flat_type                    0
block                        0
street_name                  0
storey_range                 0
floor_area_sqm               0
flat_model                   0
lease_commence_date          0
resale_price                 0
Tranc_Year                   0
Tranc_Month                  0
mid_storey                   0
lower                        0
upper                        0
mid                          0
full_flat_type               0
address                      0
floor_area_sqft              0
price_per_sqft               0
hdb_age                      0
max_floor_lvl                0
year_completed               0
residential                  0
commercial                   0
market_hawker                0
multistorey_carpark          0
precinct_pavilion            0
total_dwelling_units         0
1room_sold                   0
2room_sold                   0
3room_so

In [18]:
# Replace boolean columns

# List of columns to replace
columns_to_replace = ["residential", "commercial", "market_hawker", "multistorey_carpark", "precinct_pavilion"]

# Replace 'N' with 0, and 'Y' with 1
df[columns_to_replace] = df[columns_to_replace].replace(["N", "Y"], [0, 1])

In [19]:
# Convert all column names to be lowercase
df.columns = df.columns.str.lower()

In [20]:
# Check data type of variables
df.dtypes

id                             int64
tranc_yearmonth               object
town                          object
flat_type                     object
block                         object
street_name                   object
storey_range                  object
floor_area_sqm               float64
flat_model                    object
lease_commence_date            int64
resale_price                 float64
tranc_year                     int64
tranc_month                    int64
mid_storey                     int64
lower                          int64
upper                          int64
mid                            int64
full_flat_type                object
address                       object
floor_area_sqft              float64
price_per_sqft               float64
hdb_age                        int64
max_floor_lvl                  int64
year_completed                 int64
residential                    int64
commercial                     int64
market_hawker                  int64
m

The data type of postal should be numerical instead of object. We need to find out what data is causing postal to be object type. 

In [21]:
# Find the column index of postal
df.columns.get_loc("postal")

42

In [22]:
# Attempt to convert the postal column to integer and catch errors, and return the values of the 'error'
try:
    df["postal"] = df["postal"].astype(int)
except ValueError as e:
    error_indices = [i for i, value in enumerate(df["postal"]) if not str(value).isdigit()]
    print (df.iloc[error_indices,42])

880       NIL
3030      NIL
5793      NIL
7849      NIL
9804      NIL
14007     NIL
43215     NIL
48780     NIL
51263     NIL
51951     NIL
52403     NIL
54179     NIL
57224     NIL
61100     NIL
62338     NIL
64384     NIL
66613     NIL
75486     NIL
75494     NIL
76656     NIL
76787     NIL
82841     NIL
87213     NIL
104383    NIL
105835    NIL
107511    NIL
108757    NIL
119777    NIL
126400    NIL
129756    NIL
131736    NIL
132994    NIL
144514    NIL
Name: postal, dtype: object


postal column has some "NIL" values. This accounts for 0.02% of the dataframe, hence the decision is to drop the rows with "NIL" values.

In [23]:
# Drop rows where the postal is "NIL" and convert the postal code to be integer type.
df = df[df["postal"] != "NIL"]
df["postal"] = df["postal"].astype(int)

In [24]:
# Check that data type of postal is changed to integer
df["postal"].dtype

dtype('int64')

In [25]:
# Summary statistics of cleaned dataset
df.describe()

,id,floor_area_sqm,lease_commence_date,resale_price,tranc_year,tranc_month,mid_storey,lower,upper,mid,floor_area_sqft,price_per_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,latitude,longitude,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_nearest_distance,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
count,149742.000000,149742.000000,149742.000000,1.497420e+05,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.0,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.00000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000,149742.000000
mean,92855.241362,97.222615,1992.315576,4.486662e+05,2016.466763,6.586923,8.263173,7.226523,9.299822,8.263173,1046.504227,433.489815,28.684424,15.108012,1990.788677,1.0,0.191890,0.000100,0.000280,0.000755,124.599411,0.13964,1.766358,39.293558,49.921846,26.296637,5.975999,0.024115,0.431235,0.063990,0.626090,0.059603,0.000341,553581.399100,1.366179,103.839328,664.426314,0.489883,1.796223,5.197186,1167.571392,0.490397,1.409912,3.802580,48.704432,58.310133,764.154674,0.545071,0.261309,1.365614,103.839342,115.201164,1.366187,103.839308,394.967802,55.301031,0.113235,1.366217,103.839046,507.652819,210.125095,0.031902,1.365940,103.839274
std,53639.094344,24.441882,12.042246,1.432380e+05,2.742201,3.339657,5.482516,5.489519,5.481949,5.482516,263.092422,104.177214,12.042246,6.195127,12.103972,0.0,0.393788,0.010008,0.016745,0.027460,58.317803,5.96565,11.688068,68.868183,45.240904,33.491971,19.183276,1.138859,5.212640,2.034205,11.226896,0.669194,0.024377,187513.206992,0.042871,0.072514,368.678327,0.729694,1.421096,3.470181,1071.120967,0.767238,1.711613,4.009482,19.152557,55.515556,429.109023,0.497966,0.439350,0.042577,0.072591,55.568108,0.042886,0.072527,234.207186,17.879847,0.316881,0.042692,0.072642,308.813798,20.005456,0.175739,0.042622,0.072640
min,1.000000,31.000000,1966.000000,1.500000e+05,2012.000000,1.000000,2.000000,1.000000,3.000000,2.000000,333.684000,205.108901,3.000000,2.000000,1949.000000,1.0,0.000000,0.000000,0.000000,0.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50004.000000,1.270380,103.685228,34.265581,0.000000,0.000000,0.000000,1.873295,0.000000,0.000000,0.000000,0.000000,0.000000,21.971410,0.000000,0.000000,1.265982,103.697143,9.001285,1.270759,103.684206,45.668324,20.000000,0.000000,1.274962,103.687724,38.913475,188.000000,0.000000,1.276029,103.687207
25%,46233.250000,75.000000,1984.000000,3.460000e+05,2014.000000,4.000000,5.000000,4.000000,6.000000,5.000000,807.300000,364.676062,20.000000,12.000000,1983.000000,1.0,0.000000,0.000000,0.000000,0.000000,90.000000,0.00000,0.000000,0.000000,14.000000,0.000000,0.000000,0.000000,0.000000

## Feature Engineering

According to our research:
- district code, based on postal sector (first 2 digits of postal codes), is used as a key reference point in real estate industry to identify the locations of the property. As such, we created a new variable "postal_sector" based on the postal column. 
- the nearer the flat is to CBD, the higher the resale price. Hence, we added a new variable "dist_CBD" for each resale flat's distance to CBD, taking Raffles Place MRT as the CBD reference point.

Source: 
<br>[Ming Property](https://www.mingproperty.sg/singapore-district-code/#:~:text=They%20are%20based%20on%20the,property%20is%20in%20District%2010)
<br> [NTU](https://dr.ntu.edu.sg/handle/10356/147773)

In [26]:
# Convert the postal column to strings then apply zfill(6) to pad the postal codes with leading zeros
df["postal"] = df["postal"].astype(str).str.zfill(6)

# Extract the first 2 digits of each postal code and create a new column ‘Postal Sector’
df["postal_sector"] = df["postal"].str[:2]

In [27]:
df["postal_sector"].dtype

dtype('O')

In [28]:
# Calculate the distance between each resale flat and CBD (taking Raffles Place MRT as the reference point)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d * 1000 # Distance in m

# Distance between each resale flat and Raffles Place mrt in km
df['dist_CBD'] = df.apply(lambda row: haversine(row['latitude'], row['longitude'], 1.283931, 103.851461), axis=1)

In [29]:
# Show the information of the dataset with new variables
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149742 entries, 0 to 150633
Data columns (total 80 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         149742 non-null  int64  
 1   tranc_yearmonth            149742 non-null  object 
 2   town                       149742 non-null  object 
 3   flat_type                  149742 non-null  object 
 4   block                      149742 non-null  object 
 5   street_name                149742 non-null  object 
 6   storey_range               149742 non-null  object 
 7   floor_area_sqm             149742 non-null  float64
 8   flat_model                 149742 non-null  object 
 9   lease_commence_date        149742 non-null  int64  
 10  resale_price               149742 non-null  float64
 11  tranc_year                 149742 non-null  int64  
 12  tranc_month                149742 non-null  int64  
 13  mid_storey                 14

In [30]:
# Export cleaned dataframe as .csv
df.to_csv("../datasets/cleaned_train.csv",index=False)

[Click to proceed to next notebook for exploratory data analysis](02_EDA_final.ipynb)